# Adding AI with Models

## Navigating to the right directory

In [1]:
pwd

'/home/pranshumaan'

In [2]:
cd /media/pranshumaan/TOSHIBA\ EXT

/media/pranshumaan/TOSHIBA EXT


In [3]:
cd Dev/Langchain/

/media/pranshumaan/TOSHIBA EXT/Dev/Langchain


In [4]:
ls

1_LangChain_Quickstart.ipynb            hugging_face_api_token.txt
2_Loader_embedding_and_vector_db.ipynb  open_ai_api_key.txt
Andrej_Karpathy_Resume.pdf              the-need-to-read.txt
db/


## Importing modules and setting up environment variables

In [5]:
import os
import textwrap
from getpass import getpass

import huggingface_hub
import langchain
import openai
from langchain import HuggingFaceHub, LLMChain, PromptTemplate
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.prompts.chat import (
    AIMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [6]:
%load_ext watermark

In [7]:
%watermark

Last updated: 2023-06-08T13:35:32.037663+05:30

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.12.0

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 6.2.6-76060206-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit



In [8]:
with open('open_ai_api_key.txt', 'r') as file:
    key = file.read().rstrip()
os.environ["OPENAI_API_KEY"] = key

In [9]:
with open('hugging_face_api_token.txt', 'r') as file:
    key = file.read().rstrip()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = key

## Large Language Models

In [10]:
template = """Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])

question = (
    "How would a 5 year old at Disneyland explain derivative pricing?"
)

print(prompt.format(question=question))

Question: How would a 5 year old at Disneyland explain derivative pricing?

Answer:


### Grabbing a hugging face model: flan_alpaca

In [11]:
repo_id = "declare-lab/flan-alpaca-large"
flan = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0, "max_length": 64}
)

response = flan(prompt.format(question=question))
print(response)

A 5 year old at Disneyland would explain derivative pricing by saying that when a company makes a product, it sells it to another company. This means that when the company makes a new product, it has to pay a higher price than the original product. This is because the new product is more


In [12]:
text_davinci_003 = OpenAI(temperature=0)
response = text_davinci_003(prompt.format(question=question))
print(response.strip())

A 5 year old at Disneyland would likely not be able to explain derivative pricing. Derivative pricing is a complex financial concept that requires a certain level of financial literacy and understanding of the markets.


### Multiple Completions

In [13]:
text_davinci_003 = OpenAI(temperature=0.4, n=3, best_of=3)
response = text_davinci_003.generate([prompt.format(question=question)])
response

LLMResult(generations=[[Generation(text=' A 5 year old at Disneyland would not be able to explain derivative pricing. Derivative pricing is a complex financial concept that requires a deep understanding of mathematics, economics, and finance.', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text=' A 5 year old at Disneyland would likely not be able to explain derivative pricing. Derivative pricing is a complex financial concept that involves understanding of calculus and other advanced mathematics.', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text=' A 5 year old at Disneyland would likely not be able to explain derivative pricing. Derivative pricing is a complex financial concept that requires a good understanding of financial markets and concepts.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 18, 'total_tokens': 126}, 'model_name': 'text-da

In [14]:
for i, generation in enumerate(response.generations[0]):
    print(f"Response {i + 1}\n")
    print(generation.text.strip())
    print()

Response 1

A 5 year old at Disneyland would not be able to explain derivative pricing. Derivative pricing is a complex financial concept that requires a deep understanding of mathematics, economics, and finance.

Response 2

A 5 year old at Disneyland would likely not be able to explain derivative pricing. Derivative pricing is a complex financial concept that involves understanding of calculus and other advanced mathematics.

Response 3

A 5 year old at Disneyland would likely not be able to explain derivative pricing. Derivative pricing is a complex financial concept that requires a good understanding of financial markets and concepts.



## Embeddings

In [19]:
text = """
I grew up in a small town with a small library. The next town over had what I thought at the time was a big library, but it was actually more like my town had a tiny library, and the next one over had an actual small library. When I left to go to University, I found out what a real library looked like, and I was mesmerized. Books! Lots of books, many of them written in the current decade. My grades probably suffered from the amount of time I spent in the library reading things that didn’t directly relate to my classes. But there was one thing I found that would turn out to be life-changing: A real computer magazine. Last month, Harry McCracken pointed out that the last two widely-distributed American consumer computer magazines ceased paper publication. It is the end of an era, although honestly, it is more like a comatose patient expiring than a shocking and sudden demise.
Dr. Dobb’s first issue was far from the slick commercial magazine it would become.

Actually, before I had gone to college, I did have a subscription to Kilobaud, and I still have some copies of those. No offense to Wayne Green, but Kilobaud wasn’t that inspiring. It was more an extension of his magazine “73”, and while I enjoyed it, it didn’t get me dreaming. Dr. Dobb’s Journal — the magazine I found in the stacks of my University’s library — was tangibly different. There was an undertone of changing the world. We weren’t sure why yet, but we knew that soon, everyone would have a computer. Maybe they’d balance their checkbook or store recipes. A few people already saw the potential of digital music reproduction, although, I must admit, it was so poor at the time, I couldn’t imagine who would ever care.

I say it was life-changing to discover the few issues of Dr. Dobb’s that were published back then because I would go on to contribute to Dr. Dobb’s throughout its storied history. I wrote the infamous DOS extender series, produced special issues, and, when it went mostly digital, was the embedded system blogger for them for more years than I care to admit. In fact, I have the dubious distinction of having the final blog posted; although the website has suffered enough bit rot, I’m not sure any of it has survived other than, maybe, on the Wayback machine. While I wasn’t with the magazine for its entire 38-year run, I read it for at least 35 and had some function there for about 24 of those.
$2 bought a lot of computer magazine in 1977!

Like a lot of nostalgia, you miss it, but it isn’t coming back. You can argue for or against it, but it doesn’t really matter.

What I think Harry got right his thought piece, though, is that while digital distribution and access are awesome — look at Hackaday, for example — the magazine industry failed to figure out how to stay solvent. McCracken points out that substantial ad revenue fueled huge test labs and hardline investigative journalism in addition to the physical printing. Don’t forget that old-school magazines never made money from your subscriptions either, it was all in the ads. Now, it is hard to generate that kind of revenue and, therefore, to provide those kinds of services. So even if you prefer digital — I do — the big money is mostly gone and gone forever, it seems.
Hard to imagine if the lads needed a day job

Imagine a world where music is easily distributed for free. You’d hear a lot more music, and, in fact, you do today because independent artists can easily reach listeners directly. But how would the Beatles catalog be different if George, Paul, John, and Ringo needed day jobs? You’d imagine they might produce less and maybe even very different music.

Then again, day jobs may be in short supply soon, too, at least the ones a bunch of young musicians might take. Order taker at a fast food restaurant? That’ll be AI soon enough. Fry cook? That’s already probably a robot arm.

You have to wonder if the destruction of traditional media — TV, movie theaters, magazines, newspapers, books, and record labels — will lead to a different way to compensate creators fairly or if the jobs related to creation will just eventually dwindle to nothing but volunteers. Either way, the days of the big computer magazine is gone, just like the days you could make a living harvesting ice.

"""

Source: https://hackaday.com/2023/06/01/farewell-american-computer-magazines/

In [21]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [22]:
embedding = hf_embeddings.embed_query(text)

In [23]:
len(embedding)

384

### Chat Models

In [25]:
chat_gpt = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
messages = [
    SystemMessage(content="You were a writer on the TV show The Office."),
    HumanMessage(content="What is the relationship between Jim and Dwight?"),
]
response = chat_gpt(messages)
print(response.content)

Jim and Dwight have a complex relationship throughout the series. At the beginning, they are bitter rivals, constantly pranking and undermining each other. However, as the series progresses, they develop a mutual respect and even a friendship. They still play pranks on each other, but they also have each other's backs when it counts. Jim even becomes Dwight's best man at his wedding. Overall, their relationship is a mix of competition, humor, and genuine affection.
